# Защита персональных данных клиентов

<b>Задача:</b> Разработать метод преобразования данных, благодаря которому будет сложно восстановить персональную информацию, при этом качество моделей машинного обучения не должно измениться. Проверка качества должна быть проведена с использованием модели линейной регрессии, написанной самостоятельно и обоснованной математически. 

---

In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


---

## Первичный анализ

In [2]:
print(); print('Размер датасета:', data.shape[0], 'записей на', data.shape[1], 'признаков')
print('Обнаружено пропусков:', data.isna().sum().sum())
print('Обнаружено дубликатов:', data.duplicated().sum()); data = data.drop_duplicates()

features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

print(); print(data.head())


Размер датасета: 5000 записей на 5 признаков
Обнаружено пропусков: 0
Обнаружено дубликатов: 153

   Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
0    1     41.0   49600.0            1                  0
1    0     46.0   38000.0            1                  1
2    0     29.0   21000.0            0                  0
3    0     21.0   41700.0            2                  0
4    1     28.0   26100.0            0                  0


<b>Итог:</b> Проведя первичный анализ на маленьком датасете, не было обнаружено пропусков, но были выявлены и удалены 153 дубликата. В дальнейшем, избавление от полностью идентичных записей поможет модели машинного обучения избежать переобучения.

---

## Вывод формул

#### Линейная регрессия

Задача обучения: $w = \arg\min_w MSE(Xw, y)$, минимизировать разницу между предсказанием и целевым признаком, следовательно $Xw = y$. Нам нужно найти вектор весов, чтобы расчитать важность признака. Другими словами, неважный признак будет иметь меньший вес и влиять меньше на так называемый target. Для нахождения $w$ вначале надо домножить обе части уравнения на $X^{-1}$:

$$X^{-1}Xw=X^{-1}y$$

Вспоминая, что $XX^{-1} = E$ (единичная матрика), а $Ew = w$, то получаем:

$$w = X^{-1}y$$

С точки зрения математики мы получили верное уравнение, но на практике не найдется решения, если $X$ не является квадратной матрицей (mxm), что стремится к нулю. Но домножив уравнение на обратную матриуц, мы получим так необходимую квадратную матрицу:

$$X^TXw=X^Ty$$

Теперь при $w$ стоит множетель, от которого нужно избавиться. Для этого надо умножить обе части уравнения на этот множитель, но обратный:

$$(X^TX)^{-1}(X^TX)w=(X^TX)^{-1}X^Ty$$

Матрица умноженная на обратную матрицу исчезает и остается вектор весов, ради которого все затевалось.

$$w=(X^TX)^{-1}X^Ty$$

#### Алгоритм преобразования данных

Мы умножаем признаки размерностью $m$x$n$ на случайно сгенерированную обратимую матрицу размером $n$x$n$, а после обучения получаем предсказания $a'$ не отличимые от предсказаний без кодирования $a$. Значит надо доказать, что:

$$a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w' = a'$$

Умноженная матрица будет иметь вид $X \times P$. Тогда фомула весов примет следующий вид:

$$a' = (XP)w'$$

$$w = (X^T X)^{-1} X^T y \Rightarrow$$ 

$$w' = ((XP)^T XP)^{-1} (XP)^T y$$

Получилась формула весов, которую мы подставляем в выведенную формулу предсказаний:

$$w' = (X^T P^T XP)^{-1} P^T X^T y$$

$$a' = XP(X^T P^T XP)^{-1} P^T X^T y$$

Перемножать напрямую матрицы не стоит, поскольку для работы с формулой ее ответы должны быть в виде квадратной матрицы, поэтому сократим лишнее и получим:

$$a' = XPP^{-1}(X^T X)^{-1} P^{T^{-1}}P^T X^T y \Rightarrow$$ 

$$a' = X(X^T X)^{-1} X^T y$$

Из формулы следует, что при умножении признаков на случайно сгенерированную обратимую матрицу размером $n$x$n$, мы получим такие же предсказания, как и без умножения, но данные клиента будут зашифрованы:

$$a = Xw = (XP)w' = X(X^T X)^{-1} X^T y = a'$$

#### Гребневая регрессия (L2-регуляризация)

Иногда получается, что детерминант матрицы равен нулю. Безусловно, шанс этого стремится к нулю, но все же, ему не равен. Если такое происходит, то линейная регрессия выдаст ошибку из-за невозможности рассчитать обратную матрицу. Для избежания этой ошибки к изначальной матрице прибавляется единичная матрица, что помогает избежать сразу двух проблем: нулевого детерминанта и линейной зависимости признаков (линейная зависимость признаков весьма нежелательна в модели линейной регрессии):

$$\begin{pmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{pmatrix} + \begin{pmatrix} 1 & 2 & 0 \\ 1 & 2 & 0 \\ 1 & 2 & 0 \end{pmatrix} = \begin{pmatrix} 2 & 2 & 0 \\ 1 & 3 & 0 \\ 1 & 2 & 1 \end{pmatrix}$$

Исходя из этого, в новой формуле весов для обучения модели прибавляется единичная матрица:

$$w=(X^TX+\lambda I)^{-1}X^Ty$$

#### Метрика оценки R2

Для расчета точности модели, мы возьмем метрику R2, которая покажет насколько наша модель лучше, чем постоянная базовая линия.

$$R^2 = 1 - \frac {\sum^{n}_{i=1}(y_i - \hat{y}_i)^2} {\sum^{n}_{i=1}(y_i - \bar{y})^2}$$

$$\sigma^2 = \sum^n_{i=1} \frac {(\mu - x_i)^2}{n}$$

---

## Доказательство

In [3]:
# метрика r2 написанная вручную
def r2 (target, predictions):
    return 1 - (np.mean((target - predictions) ** 2) / np.var(target))

---

### До шифрования

In [4]:
class LinearRegression:
    
    def fit(self, features, target):
        X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
        y = target
        w = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X) + np.eye(features.shape[1] + 1)), X.T), y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, features):
        return np.dot(features, self.w) + self.w0

In [5]:
model = LinearRegression(); model.fit(features, target)
print('Качество модели:', r2(target, model.predict(features)))

Качество модели: 0.43019523064161735


---

### После шифрования

In [6]:
def encryption (X):
    
    protective_matrix = np.random.randint(1, 10, (4,4))
    encrypted_matrix = np.dot(X, protective_matrix)
    
    return encrypted_matrix

features = encryption(features)

In [7]:
model = LinearRegression(); model.fit(features, target)
print('Качество модели:', r2(target, model.predict(features)))

Качество модели: 0.43019523695347195


---

<b>Вывод:</b> Были написаны вручную и обоснованы математически три алгоритма. Доказано, что данные до шифрования и после имеют одинаковые значения. И получена метрика 0.43, которая свидетельствует о превосходстве модели над константной.